<a href="https://colab.research.google.com/github/d1m3j1/Biweekly-challenge-2/blob/master/extract_dataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import pandas as pd
from textblob import TextBlob

def read_json(json_file: str)->list:
    """
    json file reader to open and read json files into a list
    Args:
    -----
    json_file: str - path of a json file
    
    Returns
    -------
    length of the json file and a list of json
    """
    
    tweets_data = []
    for tweets in open(json_file,'r'):
        tweets_data.append(json.loads(tweets))
    
    
    return len(tweets_data), tweets_data

class TweetDfExtractor:
    """
    this function will parse tweets json into a pandas dataframe
    
    Return
    ------
    dataframe
    """
    def __init__(self, tweets_list):
        
        self.tweets_list = tweets_list

    # an example function
    def find_statuses_count(self)->list:
        statuses_count = []
        for i in self.tweets_list:
          statuses_count.append(i.get('user').get('statuses_count'))
        return statuses_count     

    def find_full_text(self)->list:
        text = []
        for i in self.tweets_list:
          text.append(i.get('text'))
        return text
    
    def find_sentiments(self, text)->list:
        polarity = []
        subjectivity = []
        for i in text:
          blob = TextBlob(i)
          sentiment = blob.sentiment
          polarity.append(sentiment.polarity)
          subjectivity.append(sentiment.subjectivity)

        return polarity, subjectivity

    def find_created_time(self)->list:
      created_at = []
      for i in self.tweets_list:
        created_at.append(i.get('created_at'))
      return created_at

    def find_source(self)->list:
        source = []
        for i in self.tweets_list:
          source.append(i.get('source'))

        return source

    def find_screen_name(self)->list:
        screen_name = []
        for i in self.tweets_list:
          screen_name.append(i.get('user').get('screen_name'))
        return screen_name

    def find_followers_count(self)->list:
        followers_count = []
        for i in self.tweets_list:
          followers_count.append(i.get('user').get('followers_count'))
        return followers_count

    def find_friends_count(self)->list:
        friends_count = []
        for i in self.tweets_list:
          friends_count.append(i.get('user').get('friends_count'))
        return friends_count
  
    def is_sensitive(self)->list:

        try:
            is_sensitive = [x.get('possibly_sensitive') for x in self.tweets_list]
        except KeyError:
            is_sensitive = None

        return is_sensitive

    def find_favourite_count(self)->list:
        favourite_count = [i.get('retweeted_status', {}).get('favorite_count', None) for i in self.tweets_list]
        return favourite_count
    
    def find_retweet_count(self)->list:
        retweet_count = [i.get('retweeted_status', {}).get('retweet_count', None) for i in self.tweets_list]
        return retweet_count

    def find_hashtags(self)->list:
        hashtags =[i.get('entities').get('hashtags') for i in self.tweets_list]

    def find_mentions(self)->list:
        mentions = [i.get('entities').get('user_mentions') for i in self.tweets_list]

    def find_lang(self):
        lang = [i.get('lang') for i in self.tweets_list]
        return lang

    def find_location(self)->list:
        location = []
        for i in self.tweets_list:
            location.append(i.get('user').get('location'))
        return location 
        
        
    def get_tweet_df(self, save=False)->pd.DataFrame:
        """required column to be generated you should be creative and add more features"""
        
        columns = ['created_at', 'source', 'original_text','polarity','subjectivity', 'lang', 'favorite_count', 'retweet_count', 
            'original_author', 'followers_count','friends_count','possibly_sensitive', 'hashtags', 'user_mentions', 'place']
        
        statuses_count = self.find_statuses_count()
        created_at = self.find_created_time()
        source = self.find_source()
        text = self.find_full_text()
        polarity, subjectivity = self.find_sentiments(text)
        lang = self.find_lang()
        fav_count = self.find_favourite_count()
        retweet_count = self.find_retweet_count()
        screen_name = self.find_screen_name()
        follower_count = self.find_followers_count()
        friends_count = self.find_friends_count()
        sensitivity = self.is_sensitive()
        hashtags = self.find_hashtags()
        mentions = self.find_mentions()
        location = self.find_location()
        data = zip(statuses_count,created_at, source, text, polarity, subjectivity,\
                   lang, fav_count, retweet_count, screen_name, follower_count, \
                   friends_count, sensitivity, hashtags, mentions, location)
        df = pd.DataFrame(data=data, columns=columns)

        if save:
            df.to_csv('processed_tweet_data.csv', index=False)
            print('File Successfully Saved.!!!')
        
        return df

                
if __name__ == "__main__":
    # required column to be generated you should be creative and add more features
    columns = ['created_at', 'source', 'original_text','clean_text', 'sentiment','polarity','subjectivity', 'lang', 'favorite_count', 'retweet_count', 
    'original_author', 'screen_count', 'followers_count','friends_count','possibly_sensitive', 'hashtags', 'user_mentions', 'place', 'place_coord_boundaries']
    _, tweet_list = read_json("/content/Economic_Twitter_Data.json")
    tweet = TweetDfExtractor(tweet_list)
    tweet_df = tweet.get_tweet_df() 
